In [1]:
import json
import requests
import pandas as pd
from pandas import json_normalize
import numpy as np
from datetime import datetime
from tqdm import tqdm
# Set display options to show all columns
pd.set_option('display.max_columns', None)
from utils import *

In [4]:
get_keys()

{'X-RapidAPI-Key': 'eea8ab2aa0mshafbbd6794b784c0p100a2ajsna9c1195dbd1f',
 'X-RapidAPI-Host': 'myanimelist.p.rapidapi.com'}